# Introductory data analysis with a real world dataset

[Click here to open this notebook in your browser](https://leifwalsh.github.io/data-analysis-problem-sets/lab/index.html?path=5-data-oriented-thinking/5.1-netflix-originals/5.1.1-netflix-originals-introductory-data-analysis.ipynb)

In this notebook, we'll cover introductory Data Analysis skills that can help you analyze new datasets in real life. We've been exploring different pandas techniques in the previous problem sets. Let's see how we can apply those techniques to clean, explore, and gain insights from the "Most Watched Netflix Original Shows" dataset. Credit: [Muhmad Akmal,  Kaggle](https://www.kaggle.com/code/muhmadakmal/most-watched-netflix-original-shows-tv-time/input)

## Loading the data

Let's load the data in pandas and glance at the first few rows.

In [ ]:
import pandas as pd
df = pd.read_csv('netflix_data.csv')

Use `df.head()` to return the first n rows of a dataframe based on position. By default, `df.head()` will return the first 5 rows 


In [ ]:
df.head()

Let's say we are interested in seeing the first 8 rows. We can simply specify:

In [ ]:
df.head(8)

`df.tail()` works in a similar way. It returns the last n rows of the dataset, with the default being 5 rows. 

In [ ]:
df.tail()

You can use `df.sample()` to take a peek at random rows in the dataset. This is especially useful in cases where you don't think the first/last rows are representative of the entire dataset.

In [ ]:
df.sample(5)

I'll also briefly explain what each column represents:

`lister-item-index:` Describes the index of the show as listed on IMDB, and begins indexing at 1.

`lister-item-header:` Show Name.

`certificate:` Maturity Rating. For example, a certificate of 15 implies that content may be unsuitable for children under the age of 15. 

`runtime:` The average runtime of each episode.

`genre:` The genres associated with the show. 

`rating:` IMDB rating.

`votes:` Number of people that rated the show.

For the purposes of this exercise, let's assume that all the data in this dataset is accurate. 

## Analyzing Columns & Rows

Now that we have a good idea of what this dataset looks like, let's analyze the columns more closely! 

In this case, we can visually see the title of all 7 columns of our dataset. But in larger datasets (say with a 100 columns) we can use `df.columns` to return all column labels.


In [ ]:
df.columns

Some of these column names are little long. Luckily, we can use `df.rename` to rename any column we'd like. 

In [ ]:
#this code generates a *new dataframe* with the renamed columns from df
df.rename(columns={'lister-item-header': 'show', 'lister-item-index': 'index'})

To keep track of all the changes we're making (like renaming columns), we'll maintain two versions of our dataset! `df` will contain the original dataframe we loaded from the `pd.read_csv` call above. 

Meanwhile, we'll use `new_df` to store the cleaned version of our data. At the end, we'll compare `df` and `new_df`so you can see all the differences.

In [ ]:
#this codes *assigns* the result of the renaming operation above to new_df
new_df = df.rename(columns={'lister-item-header': 'show', 'lister-item-index': 'index'})

You can see that `df` remains the same.

In [ ]:
df.head(1)

And that the new column names are only reflected in `new_df`. 

In [ ]:
new_df.head(1)

Let's keep going with our analysis. We can use `df.shape` to return the number of rows and columns in our orginal dataframe. 

In [ ]:
df.shape

Note that `df.shape` returns a tuple, which we can actually store as two seperate variables for easy future access.

In [ ]:
rows, columns = df.shape

In [ ]:
rows

In [ ]:
columns

There are some other helpful things we can use to describe our data! 

`df.info()` prints useful information about the dataframe, including the data types found in each column.

In [ ]:
df.info()

The `Non-Null Count` tells us the number of non null values in that column.

`Dtype` tells us the data type of the values in a column: 
* lister-item-index is of type `int64`, 
* rating is of type `float64`
* the remaining  columns are of type `object`.

In other words, only two of our columns contain numerical values (lister-item-index and rating). The lister-item-header, certificate, genre, and votes columns are all strings (the default data type for strings in DataFrames is `object`).


We also get the overall memory usage of our dataframe. Side note: If you're interested in a more accurate measure of memory usage, feel free to try `df.info(memory_usage='deep')`

Small segue since we're talking about null values! There are many, many, many ways to check for null values in your dataset! Here's another approach:

In [ ]:
df.isna().any()

You could also run the following code to see _how many_ null values you have across the entire dataset. In our case, we know the total number is 0.


In [ ]:
df.isna().sum().sum()

Another useful method (and one of my favorites) is `df.describe()`. `df.describe()` generates descriptive statistics (such as total count, mean, standard deviation, min, and max) for all _numerical_ columns in your dataset by default. 

As we saw above, only the `lister-item-index` and `rating` columns in our dataframe are numerical.

In [ ]:
df.describe()

It's not particularly helpful to learn statistics about the `lister-item-index`	column- the index just identifies each row in the dataset. 


_But_ we did get some interesting analysis on the ratings across Netflix Original shows! For example, the average rating for a show was 7.72. The standard deviation was only around .71. We also learned that the max rating achieved by any show was 8.8

If you'd like, you could also specify your own percentiles for `df.describe().`

In [ ]:
percentiles = [.05, .10, .25, .50, .75, .90, .99]
df.describe(percentiles)

Let's come back to `df.describe()` after we do some more data cleaning. This method will become even more useful for us later.

## Data Cleaning

Anubha is a Netflix addict- she'd like to watch the first episode of ALL the Netflix shows in our dataframe. How many minutes will it take her to watch all of these episodes? 

Let's sum up all the runtimes in our table to get the answer. 



In [ ]:
df['runtime'].sum()

What went wrong here? We've done this before in previous problem sets! 

Remember that Python only lets us do arithmetic operations on numbers. From `df.info()`, we learned that only `lister-item-index` and `rating` are numerical columns. 

We can also directly see which columns in our dataframe can be summed by running `df.sum(numeric_only= True)`. Runtime is not one of them. 
 

In [ ]:
df.sum(numeric_only= True)

To sum runtimes, we'll need to convert all values in the runtime column into numerical values instead of a strings (we want to go from _'60 min'_ to  _60_).

Let's do this by first removing the word 'min' from each value, and then converting the result from a string to an int! One approach is to use `df.str.replace` and `.astype()`


In [ ]:
#First, remove the word 'min' from runtimes by replacing it with an empty substring
replace_mins = df['runtime'].str.replace(' min', '')

#Second, convert runtimes to ints 
runtime_as_ints = replace_mins.astype('int')
runtime_as_ints

#You could also do this all in one line by running the following:
#runtime_as_ints = df['runtime'].str.replace(' min', '').astype('int')

In [ ]:
#To actually assign runtime_as_ints to our new_df dataframe, run:
new_df['runtime'] =  runtime_as_ints

In [ ]:
#Great! If you run this cell, you'll see that all our runtime values are of type int64!
new_df['runtime']

Now... how many minutes will it take Anubha to watch the first episode of every show in our Netflix dataset?

In [ ]:
new_df['runtime'].sum()

That's a lot of minutes 😬. This example was to show that you'll have to do some data cleaning/data manipulation with pretty much every dataset you encounter. This is a relatively "easy" dataset- it has a usability rating of 10 on Kaggle. But we still had to do some work to find right answer.

Now that we've converted the runtime column into a numerical column, we can do all kinds of other fun analysis. What's the average runtime of Netflix Original shows? 

In [ ]:
new_df['runtime'].mean()

What about the max runtime?

In [ ]:
new_df['runtime'].max()

Also, let's go back to `.describe()` and see how the output changes now that runtime is a numerical column! 

In [ ]:
new_df.describe()

The mode (or most common value) is not included in `.describe()`. Let's see what the most _common_ rating is for a Netflix Orginal show. 



In [ ]:
new_df['rating'].mode()

Note that in our previous mean, max, mode calculations, we didn't actually learn _which_ show is associate with any of those values. Let's compute the max rating again, but this time, let's find out _which_ show got the max rating.


In [ ]:
max_rating = new_df['rating'].max()

new_df.loc[new_df['rating'] == max_rating]

# Exercises 

### 1. Look up your favorite Netflix Original and return its information. If it doesn't pop up in this table, feel free to pick a random show instead.

### 2. What show had the lowest ratings? What was the rating? 

### 3. Which Netflix Original is the most popular (had the most votes)? What was the number of votes?
Hint: Votes are strings here, and need to be converted to numerical values (ints)! 

Another Hint: Last time, we had to remove the word "mins" from runtime values before converting to ints. What do we need to remove from the votes values before we can convert to ints?  


In [ ]:
# Make sure to store your new votes values in new_df 

### 4. Sarah loves the Mystery genre. She's curious which shows with the word 'Mystery' in the genre column have the highest ratings. Recommend the top 3 highest rated shows in the Mystery genre to Sarah.



### 5. Leif would like to recommend shows to his 15 year old cousin, but wants to make sure that he recommends shows that are age appropriate. Return a dataframe of all the shows with a Maturity Rating less than or equal to 15.  

Hint: How should we think about handling 'PG' in this case? We could replace 'PG' with 13. 


In [ ]:
# Make sure to store your new certificates values in new_df

### 6. Compare `df.info()` and `new_df.info()`. Compare  `df.describe()` and `new_df.describe()`. You can use new_df to answer other questions you are curious about!